# Code to generate tensors for 3DCNN Head-tail classification
We use a small sample of simulated He recoil events for demonstration purposes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from numba import jit
import ROOT
from ROOT import TVector3
import torch

ModuleNotFoundError: No module named 'ROOT'

### Read test data

In [ ]:
df = pd.read_feather("data/test.feather")

### Shift events to origin of chip
3D Convolutional neural networks are translationally invariant!

In [35]:
df['col_shift'] = df['column'].apply(lambda x: x-x.min())
df['row_shift'] = df['row'].apply(lambda x: x-x.min())
df['BCID_shift'] = df['BCID'].apply(lambda x: x-x.min())

### Determine truth direction and truth cos(theta) and truth cos(phi)

In [36]:
vecs = []
for x,y,z in zip(df['truth_px_dir'],df['truth_py_dir'],df['truth_pz_dir']):
    vecs.append(TVector3(x,y,z))
df['truth_vec'] = vecs
df['truth_costheta'] = df['truth_vec'].apply(lambda x: np.cos(x.Theta()))
df['truth_cosphi'] = df['truth_vec'].apply(lambda x: np.cos(x.Phi()))

In [37]:
@jit(nopython=True)
def get_PA(data):
    uu, dd, vv = np.linalg.svd(data-np.array([data[:,0].mean(),data[:,1].mean(),data[:,2].mean()]))
    projection = (data @ vv.T).T[0]
    return projection, vv[0]

In [39]:
project = []
vecs = []
projectt = [] #truth matched
vecst = [] #truth matched
for i in tqdm(range(0,len(df))):
    test = np.concatenate([[df['x'][i].T,df['y'][i].T,df['z'][i].T]]).T
    test_truth = np.concatenate([[df['truth_x'][i].T,df['truth_y'][i].T,df['truth_z'][i].T]]).T
    proj, vec = get_PA(test)
    projt, vect = get_PA(test_truth)
    project.append(proj)
    vecs.append(vec) #principal axis direction
    projectt.append(projt)
    vecst.append(vect)
df['proj'] = project
df['vec'] = vecs #direction along PA
df['proj_truth'] = projectt
df['vec_truth'] = vecst

100%|██████████| 1000/1000 [00:00<00:00, 1446.10it/s]


### Randomize order so that any correlations from simulation are gone

In [40]:
np.random.seed(1)
df = df.sample(frac=1)
df['original_index'] = df.index
df.index = [i for i in range(0,len(df))]

In [41]:
for key in ['col','row','BCID']:
    nkey = '%s_new'%(key)
    df[nkey] = df['%s_shift'%(key)]
    df[nkey] = df[nkey].apply(lambda x: x.astype('uint8'))

In [42]:
df['charge_new'] = df['tot'].apply(lambda x: x.astype('uint8'))

## Make labels

We assign labels based on the phi hemisphere of the true primary recoil. Events with cos(phi) < 0 are labeled 1 and events with cos(phi) > 0 are labeled 0

In [43]:
df['label'] = 1
index = df.query('truth_cosphi > 0').index.to_numpy()
df['label'][index] = 0

/home/jeff/anaconda3/envs/root_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jeff/anaconda3/envs/root_env/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


# Create and save voxelgrids

In [44]:
def voxelize_sparse(df, dim = (34,170,34)):
    voxels = []
    for i in tqdm(range(0,len(df))):
        voxelgrid = np.zeros(dim) #treat voxel locations as indices
        for x, y, z,tot in zip(df['col_new'].iloc[i], df['row_new'].iloc[i], df['BCID_new'].iloc[i], df['charge_new'].iloc[i]):
            try:
                voxelgrid[x][y][z] += tot
            except IndexError:
                continue
        voxelgrid = voxelgrid.astype('uint8')
        #if i %10000 == 0:
        #    print(i)
        voxelgrid = torch.tensor(voxelgrid).to_sparse() #need to unsqueeze later
        torch.save((voxelgrid,df['label'].iloc[i]),'tensors/%s.pt'%(i))
    return df

In [45]:
df = voxelize_sparse(df)

100%|██████████| 1000/1000 [00:01<00:00, 758.97it/s]


### Save track info with new event ordering
We remove the raw hits to save space

In [46]:
df[[col for col in df.columns if df[col].dtype != 'O']].to_feather('data/test_tracks_only.feather')